In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

### Make the data

In [ ]:
data = np.random.randint(low=-10, high=11, size=(3000, 2))
sum_results = data.sum(axis=1, keepdims=True)

In [ ]:
# Convert to tensor
data_tensor = torch.tensor(data).float()
results_tensor = torch.tensor(sum_results).float()

# Split the data
train_data, test_data, train_results, test_results = train_test_split(
    data_tensor, results_tensor, test_size=.1)

# Convert into PyTorch Datasets
train_data = torch.utils.data.TensorDataset(train_data, train_results)
test_data = torch.utils.data.TensorDataset(test_data, test_results)

# Translate into dataloader objects
batchsize = 1
train_loader = DataLoader(train_data,
                          batch_size=batchsize,
                          shuffle=True,
                          drop_last=True)
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

In [ ]:
print(data_tensor)
print(data_tensor.shape)
print(results_tensor)
print(results_tensor.shape)

### Make the net

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.input = nn.Linear(2, 16)
        self.fc1 = nn.Linear(16, 1)
        self.output = nn.Linear(1, 1)

    # forward pass
    def forward(self, x):
        x = nn.functional.relu(self.input(x))
        x = nn.functional.relu(self.fc1(x))
        return self.output(x)

In [ ]:
class NNPipeline():

    def __init__(self, train_loader, test_loader):
        self._net = Net()
        self._lossfun = nn.MSELoss()
        self._optimizer = torch.optim.Adam(self._net.parameters(), lr=.01)
        self._train_loader = train_loader
        self._test_loader = test_loader

    def train(self, num_epochs=100):
        losses = torch.zeros(num_epochs)
        train_accuracy = []
        test_accuracy = []

        for epochi in range(num_epochs):
            batch_loss = []
            for X, y in self._train_loader:
                # forward pass and loss
                y_hat = self._net(X)
                loss = self._lossfun(y_hat, y)

                # backprop
                self._optimizer.zero_grad()
                loss.backward()
                self._optimizer.step()

                # loss from this batch
                batch_loss.append(loss.item())

            # and get average losses across the batches
            losses[epochi] = np.mean(batch_loss)

            # compute train accuracy
            X, y = next(iter(self._train_loader))
            with torch.no_grad():
                y_hat = self._net(X)
            train_accuracy.append(100 * torch.mean(
                (torch.abs(y_hat - train_results) < 1).float()))

            # compute test accuracy
            X, y = next(iter(self._test_loader))
            with torch.no_grad():
                y_hat = self._net(X)
            test_accuracy.append(100 * torch.mean(
                (torch.abs(y_hat - test_results) < 1).float()))

        return train_accuracy, test_accuracy, losses, self._net

    def predict(self, loader):
        with torch.no_grad():
            y_hat = self._net(loader.dataset.tensors[0])
        return torch.round(y_hat.detach())

### Test the net

neural_network_dev = NNPipeline(train_loader, test_loader)
train_accuracy_dev, test_accuracy_dev, losses_dev, net_dev = neural_network_dev.train(num_epochs=10)
print(f'Final training accuracy: {train_accuracy_dev}')
print(f'Final testing accuracy: {test_accuracy_dev}')

neural_network_dev.predict(test_loader)

### Run the experiment

In [ ]:
trained_models = []
for trained_model_index in range(10):
    print(f"Training model {trained_model_index}")
    pipeline = NNPipeline(train_loader, test_loader)
    train_accuracy, test_accuracy, losses, net = pipeline.train(
        num_epochs=10)
    trained_models.append({
        "train_accuracy": train_accuracy,
        "test_accuracy": test_accuracy,
        "losses": losses,
        "net": net,
        "pipeline": pipeline
    })

In [ ]:
for model in trained_models:
    print(model["test_accuracy"])

In [ ]:
test_predictions = trained_models[-1]["pipeline"].predict(test_loader)

In [ ]:
test_predictions

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.plot(test_loader.dataset.tensors[1], 's')
plt.plot(test_predictions, 'rx')
plt.legend(['True sum','Predicted sum'])
plt.xlabel('Sample index')
plt.ylabel('Sum')
plt.title('Predicted vs. actual sum')
plt.show()